**Eliteserien**

Bibilioteker og drive

In [18]:
from scipy.stats import poisson
import pandas as pd
df = pd.read_csv("/content/matches.csv")

In [19]:
df.head()

,match_id,team,opponent,xG_non_penalty,xG_non_penalty_against,score,score_against
0,1,Bodø/Glimt,Bryne,2.55,0.40,5,0
1,2,Brann,KFUM,1.19,0.64,1,1
2,1,Bryne,Bodø/Glimt,0.40,2.55,0,5
3,3,FK Haugesund,Molde,0.65,2.16,0,2
4,4,Fredrikstad,Sarpsborg 08,1.24,1.07,2,0


In [20]:
# 🧩 Funksjon for sannsynligheter, flagg og expected points
def add_probabilities(df):
    df = df.copy()

    # 🔢 Differanse og indikatorer
    df["npxG_diff"] = df["xG_non_penalty"].astype(float) - df["xG_non_penalty_against"].astype(float)
    df["npxG_advantage"] = (df["npxG_diff"] >= 0.75).astype(int)
    df["npxG_disadvantage"] = (df["npxG_diff"] <= -0.75).astype(int)

    # 🧮 Poisson-basert sannsynlighet
    def poisson_probs(for_xg, against_xg):
        max_goals = 10
        home_probs = [poisson.pmf(i, for_xg) for i in range(max_goals)]
        away_probs = [poisson.pmf(j, against_xg) for j in range(max_goals)]

        win = sum(home_probs[i] * sum(away_probs[:i]) for i in range(max_goals))
        draw = sum(home_probs[i] * away_probs[i] for i in range(max_goals))
        loss = 1 - win - draw
        return win, draw, loss

    win_probs = []
    draw_probs = []
    loss_probs = []
    clean_sheet_probs = []

    for _, row in df.iterrows():
        for_xg = float(row["xG_non_penalty"])
        against_xg = float(row["xG_non_penalty_against"])

        win, draw, loss = poisson_probs(for_xg, against_xg)
        clean_sheet = poisson.pmf(0, against_xg)

        win_probs.append(round(win, 3))
        draw_probs.append(round(draw, 3))
        loss_probs.append(round(loss, 3))
        clean_sheet_probs.append(round(clean_sheet, 3))

    df["P_win_%"] = win_probs
    df["P_draw_%"] = draw_probs
    df["P_loss_%"] = loss_probs
    df["P_clean_sheet_%"] = clean_sheet_probs
    df["expected_points"] = round(df["P_win_%"] * 3 + df["P_draw_%"] * 1, 2)
    df["clean_sheet_flag"] = (df["score_against"] == 0).astype(int)
    df["opponent_clean_sheet_flag"] = (df["score"] == 0).astype(int)
    df["low_npxG_flag"] = (df["xG_non_penalty"].astype(float) < 1.0).astype(int)
    df["opponent_low_npxG_flag"] = (df["xG_non_penalty_against"].astype(float) < 1.0).astype(int)


    return df

In [21]:
# Kjører funksjon på datasett
df_with_probs = add_probabilities(df)

In [22]:
# Viser noen rader for kontroll
df_with_probs.head()

,match_id,team,opponent,xG_non_penalty,xG_non_penalty_against,score,score_against,npxG_diff,npxG_advantage,npxG_disadvantage,P_win_%,P_draw_%,P_loss_%,P_clean_sheet_%,expected_points,clean_sheet_flag,opponent_clean_sheet_flag,low_npxG_flag,opponent_low_npxG_flag
0,1,Bodø/Glimt,Bryne,2.55,0.40,5,0,2.15,1,0,0.839,0.121,0.041,0.670,2.64,1,0,0,1
1,2,Brann,KFUM,1.19,0.64,1,1,0.55,0,0,0.493,0.308,0.199,0.527,1.79,0,0,0,1
2,1,Bryne,Bodø/Glimt,0.40,2.55,0,5,-2.15,0,1,0.040,0.121,0.839,0.078,0.24,0,1,1,0
3,3,FK Haugesund,Molde,0.65,2.16,0,2,-1.51,0,1,0.098,0.179,0.722,0.115,0.47,0,1,1,0
4,4,Fredrikstad,Sarpsborg 08,1.24,1.07,2,0,0.17,0,0,0.401,0.282,0.317,0.343,1.48,1,0,0,0
